## Modules à importer

In [1]:
import math
import scipy
import pickle
import numpy as np
import pandas as pd
import iminuit as im
import ipywidgets as ipw
from importlib import reload
from scipy.stats import poisson
from variaIa import baserate
from variaIa import stretchevol
from astropy.cosmology import Planck15 as cdata
from IPython.display import display, Latex
import matplotlib.pyplot as plt
import random

## Définitions pandas, surveys, $z[:z_{\text{max}}]$, $x_1[:z_{\text{max}}]$, mean, std et données totales

In [4]:
cons = ipw.Checkbox(
    value=False,
    description='Conservative')
display(cons)

Checkbox(value=False, description='Conservative')

In [12]:
d = pd.read_csv('../Data/data_cheat.csv', sep=' ', index_col = 'CID')
d_snf = pd.read_csv('../Data/lssfr_paper_full_sntable.csv',sep=',')

surveys = ['SNF', 'SDSS', 'PS1', 'SNLS', 'HST']

if cons.value:
    surv = {'SNF':  d_snf.loc[d_snf['name'].str.contains('SNF|LSQ|PTF',na=False,regex=True)],
            'SDSS': d[d['IDSURVEY'] == 1],
            'PS1':  d[d['IDSURVEY'] == 15],
            'SNLS': d[d['IDSURVEY'] == 4],
            'HST':  d[d['IDSURVEY'].isin([101, 100, 106])]}
else:
    surv = {'SNF':  d_snf,
            'SDSS': d[d['IDSURVEY'] == 1],
            'PS1':  d[d['IDSURVEY'] == 15],
            'SNLS': d[d['IDSURVEY'] == 4],
            'HST':  d[d['IDSURVEY'].isin([101, 100, 106])]}

with open('../Data/zmax_mlim', 'rb') as f:
    z_max = pickle.load(f)

z_max['SNF'] = [10, 10]
z_max['HST'] = [10, 10]

dgmap = plt.cm.get_cmap('viridis')
colors = {'SNF': dgmap(0),
          'SDSS': dgmap(50),
          'PS1': dgmap(125),
          'SNLS': dgmap(200),
          'HST': dgmap(300)}

zmax_cuts = dict()
z_zcuts = dict()
x1_zcuts = dict()
x1_err_zcuts = dict()

for survey in surveys[1:]:
    zmax_cuts[survey] = np.where(surv[survey].zCMB.values < z_max[survey][1])
    z_zcuts[survey] = surv[survey].zCMB.values[zmax_cuts[survey]]
    x1_zcuts[survey] = surv[survey].x1.values[zmax_cuts[survey]]
    x1_err_zcuts[survey] = surv[survey].x1ERR.values[zmax_cuts[survey]]
    
zmax_cuts['SNF'] = np.where(surv['SNF']['host.zcmb'].values < z_max['SNF'][1])
z_zcuts['SNF'] = surv['SNF']['host.zcmb'].values[zmax_cuts['SNF']]
x1_zcuts['SNF'] = surv['SNF']['salt2.X1'].values[zmax_cuts['SNF']]
x1_err_zcuts['SNF'] = surv['SNF']['salt2.X1.err'].values[zmax_cuts['SNF']]

datax_all = np.concatenate(
    (np.concatenate(
        (np.concatenate(
            (np.concatenate((x1_zcuts['SNF'],
                             x1_zcuts['SDSS'])),
             x1_zcuts['PS1'])),
         x1_zcuts['SNLS'])),
     x1_zcuts['HST']))
                                                     
datax_err_all = np.concatenate(
    (np.concatenate(
        (np.concatenate(
            (np.concatenate((x1_err_zcuts['SNF'],
                             x1_err_zcuts['SDSS'])),
             x1_err_zcuts['PS1'])),
         x1_err_zcuts['SNLS'])),
     x1_err_zcuts['HST']))
                                                     
dataz_all = np.concatenate(
    (np.concatenate(
        (np.concatenate(
            (np.concatenate((z_zcuts['SNF'],
                             z_zcuts['SDSS'])),
             z_zcuts['PS1'])),
         z_zcuts['SNLS'])),
     z_zcuts['HST']))

z_mean = dict()
z_std = dict()
x1_mean = dict()
x1_std = dict()

for survey in surveys:
    z_mean[survey] = np.mean(z_zcuts[survey])
    z_std[survey] = np.std(z_zcuts[survey])
    x1_mean[survey] = np.mean(x1_zcuts[survey])
    x1_std[survey] = np.std(x1_zcuts[survey])

## Set print level

In [6]:
p_l = ipw.IntSlider(min = 0, max = 1, value = 0, description='print_level:')
display(p_l)

IntSlider(value=0, description='print_level:', max=1)

## Minimisation des modèles à mettre dans le _pandas_

In [13]:
reload(stretchevol)

########################################### Evol Simple

evol1G1M1S = stretchevol.Evol1G1M1S()
evol1G1M1S.set_data(dataz_all, datax_all, datax_err_all)

print('1G1M1S')

evol1G1M1S.minimize(p_l.value)

########################################### Evol Kessler

evol1G1M2S = stretchevol.Evol1G1M2S()
evol1G1M2S.set_data(dataz_all, datax_all, datax_err_all)

print('1G1M2S')

evol1G1M2S.minimize(p_l.value)

# evol1G1M2S.traceur()

########################################### Evol Howell

evol2G2M2S = stretchevol.Evol2G2M2S()
evol2G2M2S.set_data(dataz_all, datax_all, datax_err_all)

print('2G2M2S')

evol2G2M2S.minimize(p_l.value)
#evol2G2M2S.plotter()

########################################### Evol Howell Fixed

evol2G2M2SF = stretchevol.Evol2G2M2SF()
evol2G2M2SF.set_data(dataz_all, datax_all, datax_err_all)

print('2G2M2SF')

evol2G2M2SF.minimize(p_l.value)

########################################### Evol NR 1S

evol3G2M1S = stretchevol.Evol3G2M1S()
evol3G2M1S.set_data(dataz_all, datax_all, datax_err_all)

print('3G2M1S')

evol3G2M1S.minimize(p_l.value)

########################################### Evol NR 1S Fixed

evol3G2M1SF = stretchevol.Evol3G2M1SF()
evol3G2M1SF.set_data(dataz_all, datax_all, datax_err_all)

print('3G2M1SF')

evol3G2M1SF.minimize(p_l.value, limit_f = (0,1), limit_a = (0,1))

########################################### Evol NR 1S SNF

evol3G2M1SSNF = stretchevol.Evol3G2M1SSNF()
evol3G2M1SSNF.set_lssfr(d_snf['salt2.X1'], d_snf['salt2.X1.err'],
                        d_snf['lssfr'], d_snf['lssfr.err_down'], d_snf['lssfr.err_up'],
                        d_snf['p(prompt)'])

evol3G2M1SSNF.minimize(p_l.value, limit_a = (0,1))

# evol3G2M1SSNF.plotter("3G2M1SSNF")

########################################### Evol NR 2S

evol3G2M2S = stretchevol.Evol3G2M2S()
evol3G2M2S.set_data(dataz_all, datax_all, datax_err_all)

print('3G2M2S')

evol3G2M2S.minimize(p_l.value)

########################################### Evol NR 2S Fixed

evol3G2M2SF = stretchevol.Evol3G2M2SF()
evol3G2M2SF.set_data(dataz_all, datax_all, datax_err_all)

print('3G2M2SF')

evol3G2M2SF.minimize(p_l.value, limit_f = (0,1), limit_a = (0,1))

########################################### Evol NR 2S SNF

evol3G2M2SSNF = stretchevol.Evol3G2M2SSNF()
evol3G2M2SSNF.set_lssfr(d_snf['salt2.X1'], d_snf['salt2.X1.err'],
                        d_snf['lssfr'], d_snf['lssfr.err_down'], d_snf['lssfr.err_up'],
                        d_snf['p(prompt)'])

print('3G2M2SSNF')

evol3G2M2SSNF.minimize(p_l.value, limit_a = (0,1))

# evol3G2M2SSNF.plotter()

########################################### Evol NR 3S

evol3G3M3S = stretchevol.Evol3G3M3S()
evol3G3M3S.set_data(dataz_all, datax_all, datax_err_all)

print('3G3M3S')

evol3G3M3S.minimize(p_l.value)

########################################### Evol NR 3S Fixed

evol3G3M3SF = stretchevol.Evol3G3M3SF()
evol3G3M3SF.set_data(dataz_all, datax_all, datax_err_all)

print('3G3M3SF')

evol3G3M3SF.minimize(0, limit_f = (0,1), limit_a = (0,1))
#evol3G3M3SF.migrad_out[0]

#fig=plt.gcf()
#fig.savefig('/home/nora/Documents/Thèse/Images/3G2M2SSNF.pdf', bbox_inches='tight')

1G1M1S
1G1M2S
2G2M2S
2G2M2SF
3G2M1S
3G2M1SF
3G2M2S
3G2M2SF
3G2M2SSNF
3G3M3S
3G3M3SF


## Get proba

In [8]:
def get_proba(model):
    return np.exp((evol3G2M1S.get_aicc() - model.get_aicc())/2)

In [14]:
modèles = [evol3G2M1S, evol3G2M2S, evol2G2M2S, evol3G3M3S,
           evol2G2M2SF, evol3G2M2SF, evol3G3M3SF,
           evol1G1M2S, evol1G1M1S, evol3G2M1SF]

d_mod_comp_cons = pd.DataFrame({'Nom modèle': [type(k).__name__ for k in modèles],
                         'Description': [k.__doc__ for k in modèles],
                         'Paramètres libres': [len(k.FREEPARAMETERS) for k in modèles],
                         'ln L': [k.get_logl() for k in modèles],
                         '$\Delta$ AICc': [evol3G2M1S.get_aicc() - k.get_aicc() for k in modèles],
                         'AICc': [k.get_aicc() for k in modèles],
                         'Proba (from NR 1S)': [get_proba(k) for k in modèles]})

d_mod_comp_cons

,Nom modèle,Description,Paramètres libres,ln L,$\Delta$ AICc,AICc,Proba (from NR 1S)
0,Evol3G2M1S,Evol NR 1S,4,1477.925225,0.000000,1485.996147,1.000000e+00
1,Evol3G2M2S,Evol NR 2S,5,1477.837318,-1.947743,1487.943890,3.776183e-01
2,Evol2G2M2S,Evol Howell,4,1479.306407,-1.381182,1487.377329,5.012798e-01
3,Evol3G3M3S,Evol NR 3S,7,1476.721794,-4.925290,1490.921438,8.520925e-02
4,Evol2G2M2SF,Evol Howell Fixed,5,1484.227341,-8.337766,1494.333913,1.546953e-02
5,Evol3G2M2SF,Evol NR 2S Fixed,6,1484.227425,-10.380744,1496.376892,5.569934e-03
6,Evol3G3M3SF,Evol NR 3S Fixed,8,1478.900015,-9.161010,1495.157158,1.024972e-02
7,Evol1G1M2S,Evol Kessler,3,1589.981772,-110.028103,1596.024250,1.281448e-24
8,Evol1G1M1S,Evol 1 gaussienne,2,1521.810866,-39.835920,1525.832067,2.237381e-09
9,Evol3G2M1SF,Evol NR 1S Fixed,5,1521.810883,-45.921308,1531.917455,1.067369e-10
